In [ ]:
from os import path

import numpy as np
import matplotlib.pyplot as plt
import math

from mpl_toolkits.mplot3d import Axes3D

# import skimage
# print(skimage.__version__)
# Requires skimage version >= 0.16.2
from skimage.draw import line_nd, ellipse

from scipy.ndimage import gaussian_filter
import scipy.io

# Parameters

In [ ]:
# Output path
output_dir = path.join('..', 'data', 'volumes')

# Parameters
obj_dims = (15, 648, 486)
scale = (5, 1.5, 1.5)

N = 200
pt_sparsity = 5e-6
line_sparsity = 6e-7
min_sep = (5, 3, 3)

# Volume Generator

In [ ]:
class VolumeGenerator():
    """
    Constructs a generator to build volumes with random points and lines.
    
    Attributes:
    
        - dims: Tuple specifying (z, y, x) in pixels
        - scale: Tuple specifying size of each pixel in um (z, y, x)

        Dataset properties:
        - N: number of volumes in dset
        - dset: Array containing N volumes (n, z, y, x)
        - bead_size: Tuple specifying FWHM of bead size in um (z, y, x)
        - min_sep: Tuple specifying minimum separation in um (z, y, x)
    """
    
    
    def __init__(self, dims=(80, 648, 486), scale=(5, 1.5, 1.5)):
        """
        Inputs:
        - dims: Tuple specifying (z, y, x) in pixels
        - scale: Tuple specifying size of each pixel in um (z, y, x)
        """
        self.dims = dims
        self.scale = scale
        
    def generate(self, N, bead_size=(5, 3, 3), min_sep=(5, 3, 3), 
                     pt_sparsity=1e-5, line_sparsity=1e-6):
        """
        Generates volumes with shape (z, y, x) to store into self.dset with shape (N, z, y, x)
        
        Inputs:
        - N: Number of volumes to generate
        - bead_size: Tuple specifying bead size in um (z, y, x)
        - min_sep: Tuple specifying minimum separation in um (z, y, x)
        - pt_sparsity: Sparsity of points to be added relative to volume
        - line_sparsity: Sparsity of endpoints to be added relative to volume
        """
        self.N = N
        self.bead_size = bead_size
        self.min_sep = min_sep
        
        Z, Y, X = self.dims
        self.dset = np.zeros((N, Z, Y, X))
        
        for n in range(N):
            self.dset[n] = self._generate_volume(pt_sparsity, line_sparsity)
        
    def visualize(self, num_volumes):
        """
        Visualizes random volumes from dset.
        
        Inputs:
        - num: Number of volumes to visualize
        """
        batch = np.random.choice(self.N, num_volumes, replace=False)
        
        for i in batch:
            volume = self.dset[i]
            z, y, x = np.nonzero(volume > 0.1)
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
#             sc = ax.scatter(x, z, y, zdir='z', c=volume[z, y, x], cmap='binary')
            sc = ax.scatter(x, z, y, zdir='z', c=volume[z, y, x])

            plt.colorbar(sc)
            plt.title("volume #" + str(i))
        
        plt.show()
        
#     def save_npy(self, volume_num, output_dir=output_dir):
#         """
#         Saves a single dset[volume_num] of shape (1, z, y, x) to .npy file.
#         """
#         np.save(output_dir + '/volume' + str(volume_num), self.dset[volume_num])
        
        
#     def save_mat(self, volume_num, output_dir=output_dir):
#         """
#         Saves a single dset[volume_num] of shape (1, z, y, x) to .mat file.
#         """
#         scipy.io.savemat(output_dir + '/volume' + str(volume_num) + ".mat", dict(volume=self.dset[volume_num]))
        
        
    def save_all_npy(self, label='', output_dir=output_dir):
        """
        Saves dset of shape (n, z, y, x) in the dset to a .npy file.

        Input:
        - label: String to label dataset with
        """
        z, y, x = self.dims
        z, y, x = str(z), str(y), str(x)
        N = str(self.N)
        output_path = path.join(output_dir, 'volumes_' + N + '_' + z + '_' + y + '_' + x + '_' + str(label))
        np.save(output_path, self.dset)
        
        
    def _generate_volume(self, pt_sparsity, line_sparsity):
        """
        Generates a single volume. 
        Returns array of shape (z, y, x).
        
        Inputs:
        - pt_sparsity: Sparsity of points to be added relative to volume
        - line_sparsity: Sparsity of endpoints to be added relative to volume
        """
        Z, Y, X = self.dims
        volume = np.zeros((Z, Y, X))
        occupied = np.zeros((Z, Y, X))
        
        num_pts = int(Z*Y*X*pt_sparsity)
        num_lines = int(Z*Y*X*line_sparsity)
        
        for i in range(num_pts):
            self._generate_bead_uniform(volume, occupied)
        
        for i in range(num_lines):
            self._generate_line_uniform(volume)
        
#         fwhm_scaled = np.array(self.bead_size) / np.array(self.scale)
#         sigmas = fwhm_scaled / np.sqrt(8 * np.log(2)) 
#         volume = gaussian_filter(volume, sigmas)
        
        return volume
    
    def _generate_bead_uniform(self, volume, occupied):
        """
        Uniform randomly generates beads separated in space.
        
        Inputs:
        - volume: volume to add points to.
        - occupied: Numpy array of shape (z, y, x) to enforce min separation.
        """
        Z, Y, X = self.dims
        z = np.random.randint(0, Z)
        y = np.random.randint(0, Y)
        x = np.random.randint(0, X)
        while occupied[z, y, x] == 1:
            z = np.random.randint(0, Z)
            y = np.random.randint(0, Y)
            x = np.random.randint(0, X)
        
        volume[z, y, x] = 1
        self._occupy_bead(occupied, (z, y, x))
        
    def _occupy_bead(self, occupied, center):
        """
        Enforces min separation between beads at a center.
        
         Inputs:
        - occupied: Numpy array of shape (z, y, x) with ones to specify which points are occupied
        - center: Tuple specifying center of form (z, y, x)
        """
        Z, Y, X = self.dims
        z, y, x = center
        
        
        sep_idx = np.array(self.min_sep) / np.array(self.scale)
        z_sep, y_sep, x_sep = sep_idx
        elip = ellipse(y, x, y_sep, x_sep, shape=(Y, X))
        z_start = max(0, z - math.floor(z_sep))
        z_end = min(Z, z + math.ceil(z_sep))
        
        y_elip, x_elip = elip
        
        occupied[z_start:z_end, y_elip, x_elip] = 1
    
    def _generate_pt_uniform(self, volume, occupied):
        """
        Adds unique random point to volume. Point is uniformly distributed.
        
        Inputs:
        - volume: volume to add points to.
        - occupied: Numpy 3Z array of shape (z, y, x) with ones to specify which points are occupied
        """
        Z, Y, X = self.dims
        z = np.random.randint(0, Z)
        y = np.random.randint(0, Y)
        x = np.random.randint(0, X)
        
        while occupied[z, y, x] == 1:
            z = np.random.randint(0, Z)
            y = np.random.randint(0, Y)
            x = np.random.randint(0, X)
        
        occupied[z, y, x] = 1
        
        # choose distribution fn for magnitude
#         volume[z, y, x] = self._get_magnitude_uniform()
        volume[z, y, x] = 1
    
    def _generate_line_uniform(self, volume):
        """
        Add random line to volume. Chooses two points from uniform distribution to generate a line.
        
        Inputs:
        - volume: volume to add line to
        """
        Z, Y, X = self.dims
        z1 = np.random.randint(0, Z)
        y1 = np.random.randint(0, Y)
        x1 = np.random.randint(0, X)
        
        z2 = np.random.randint(0, Z)
        y2 = np.random.randint(0, Y)
        x2 = np.random.randint(0, X)
        
        lin = line_nd((z1, y1, x1), (z2, y2, x2), endpoint=False)
        
        # choose distribution fn for magnitude
#         volume[lin] = self._get_magnitude_uniform()
        volume[lin] = 1

    
    def _get_magnitude_uniform(self):
        """
        Returns magnitude chosen from uniform(0, 1)
        """
        return np.random.uniform()
    
    
    def _generate_line_beads(self):
        """
        Eventually, enforce line separation as if similar to beads in a row?
        """
        pass
    

# Generation and visualization

In [ ]:
vg = VolumeGenerator(dims=obj_dims, scale=scale)
vg.generate(N=N, pt_sparsity=pt_sparsity, line_sparsity=line_sparsity, min_sep=min_sep)

In [ ]:
vg.visualize(num_volumes=3)

In [ ]:
vg.save_all_npy('1')